In [3]:
import numpy as np, serial, torch, time
import torch, torch.nn as nn, torch.optim as optim, numpy as np

parm = torch.load("dyn_model.pt", map_location="cpu", weights_only=False)

net = nn.Sequential(nn.Linear(5,128), nn.ReLU(),
                    nn.Linear(128,128), nn.ReLU(),
                    nn.Linear(128,4))
net.load_state_dict(parm["net"]); net.eval()

def norm(x,mu,sig): return (x-mu)/sig
def denorm(x,mu,sig): return x*sig+mu

H   = 10      # horizon (≈0.2 s)
K   = 200     # how many random sequences to shoot
SIG = 0.3     # exploration std in action space
# ---- константы рулевого привода ----
STEER_MIN, STEER_MAX = 1968, 4004
STEER_C   = (STEER_MIN + STEER_MAX) / 2
STEER_SPAN= (STEER_MAX - STEER_MIN) / 2

def mpc_control(s_now):
    '''s_now: np.array shape (4,) in raw units'''
    best_a0, best_cost = 0.0, 1e9
    s0 = torch.tensor(norm(s_now, parm["mu_s"], parm["sig_s"])).float()

    for _ in range(K):
        # sample action sequence around 0
        a_seq = np.random.normal(0, SIG, size=(H,))
        cost  = 0.0
        s     = s0.clone()
        for a in a_seq:
            a_n = torch.tensor(norm([a],[parm["mu_a"]],[parm["sig_a"]])).float()
            s   = net(torch.cat([s, a_n]))
            s_d = denorm(s, parm["mu_sn"], parm["sig_sn"])
            yaw, ay = float(s_d[0]), float(s_d[1])
            cost += yaw*yaw + 0.5*ay*ay           # quadratic cost
        if cost < best_cost:
            best_cost, best_a0 = cost, a_seq[0]

    # convert back to PWM
    steer_pwm = int(best_a0 * STEER_SPAN + STEER_C)
    steer_pwm = int(np.clip(steer_pwm, STEER_MIN, STEER_MAX))
    return steer_pwm

# ---------- example of serial loop ----------
ser = serial.Serial("/dev/ttyUSB0", 115200)
while True:
    line = ser.readline().decode().strip()
    t,ax,ay,yr,steer = map(float, line.split(",")[:5])
    # use your own ax-world / ay-world calc or Kalman if needed
    s_now = np.array([yr, ay, ax, 0.0])   # angular_accel≈0 for first step
    steer_cmd = mpc_control(s_now)
    ser.write(f"{steer_cmd}\n".encode())

SerialException: could not open port '/dev/ttyUSB0': FileNotFoundError(2, 'Системе не удается найти указанный путь.', None, 3)